Exemple inspiré de la vidéo : https://www.youtube.com/watch?v=Uz3B9fVb4LQ

# 0. Import Python libraries

In [277]:
#system
import sys
import time
#math
import numpy as np
import sympy as sy
#data
import pandas as pd
#vis
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# 1. $f$, $\nabla f$ and $\nabla^2 f$ definition

In [278]:
"""This section enables to express f (objective function), its gradient vector and hessian matrix symbolically and evaluate them at specific coordinates.
To do so, express analititically f by indexing the symbolic variable x. For instance x[0] represent first dimension, x[1] the second and so on.
grad_f_exp and hess_f_exp are only there to automatically differentiate f_exp. To evaluate the f, grad_f or hess_f functions, call them precising the desirated coordinates.
This evaluation use lambdify function and uses "numpy" format.

    Parameters
    ----------
    x : sympy.IndexedBase
        symbolic variable managing indexes. For instance, instead of declaring n variable for the n dimensions (x, y, z, ...) which isn't easily scalable, 
        here variables are automaticly indexed (x_1, x_2, ..., x_n).

    xk : np.array (float)
        Coordinates to evaluate the function.

    Returns
    -------
    numpy.float (scalar or numpy.array)
        Depending on the considered function, return a scalar, a (n, ) vector or a (n, n) array.

    Notes
    ------
    The use of specific function to express f, grad_f and hess_f is to improve scalability in the case of not having analitic expression.
    For instance, when only have a black box model which evaluate f, grad_f (and hess_f). 

    References
    ------
    https://docs.sympy.org/latest/index.html

    Examples
    ------
    >>>x = sy.IndexedBase('x')
    >>>xk = [-4, 3]
    >>>dk = grad_f(x, xk)
    """

def f_exp(x):
    #return 3*x[0]**2 + 2*x[1]**2 + 20*sy.cos(x[0])*sy.cos(x[1])+40
    return .26*(x[0]**2 + x[1]**2) - .48*x[0]*x[1]

def f(x, xk):
    return sy.lambdify(x, f_exp(x), "numpy")(xk)

def grad_f_exp(x, xk):
    return [sy.diff(f_exp(x), x[i]) for i in range(len(xk))]

def grad_f(x, xk):
    lambdify = [sy.lambdify(x, gf, "numpy") for gf in grad_f_exp(x, xk)]
    return np.array([lambdify[i](xk) for i in range(len(xk))])

def hess_f_exp(x, xk):
    return [[sy.diff(g, x[i]) for i in range(len(xk))] for g in grad_f_exp(x, xk)]

def hess_f(x, xk):
    lambdify = [[sy.lambdify(x, gf, "numpy") for gf in Hs] for Hs in hess_f_exp(x, xk)]
    return np.array([[lambdify[i][j](xk) for i in range(len(xk))] for j in range(len(xk))])

# 2. Armijo rule

In [279]:
class Armijo():
    """Armijo is a class implementing the Armijo's rule using a backtracking method. It has 3 functions in addition to the __init__ function.
    - fit(): is the main function, ensuring the convergency and implementing the backtracking algorithm. It calls phi(), theta() 
    - phi(): is the function to evaluate the first terme of Armijo's rule. It calls the function f() previously defined by the user.
    - theta(): is the function to evaluate the second terme of Armijo's rule. It calls the functions f() and grad_f() previously defined by the user.

    Notes
    ------
    The class respect a commun pattern for all line search classes. To evaluate descent step call the function fit().

    References
    ------
    J. F. Bonnans, J. C. Gilbert, C. Lemaréchal, C. A. Sagastizábal, 2006. Numerical Optimization: Theoretical and Practical Aspects. https://doi.org/10.5860/choice.41-0357
    https://github.com/scikit-learn/scikit-learn

    Examples
    ------
    >>>armijolinesearch = Armijo(x, xk, dk, a0=a0, b=b, w_1=w_1, max_it=max_it)
    >>>m, alpha = armijolinesearch.fit()
    """
    def __init__(
        self,
        x,
        xk,
        dk,
        *,
        w_1=.015,
        a0=15,
        b=.95,
        max_it=50
        ):
        """
        Parameters
        ----------
        x : sympy.IndexedBase
            Previously defined, needed to evaluate phi() and theta()
        xk : np.array (float)
            Coordinates to evaluate the function
        dk : np.array (float)
            direction descent at iteration k
        w_1 : float, optional
            hyperparameter defining Armijo's condition, by default .015
        a0 : int, optional
            hyperparameter defining initial guess of a (biggest), by default 15
        b : float, optional
            hyperparameter defining the backtracking geometric convergency, by default .95
        max_it : int, optional
            hyperparameter defining maximum of iteration, by default 50
        """
        self.x = x
        self.xk = xk
        self.dk = dk
        self.a0 = a0
        self.b = b
        self.w_1 = w_1
        self.max_it = max_it

    def phi(Armijo, m):
        phi = f(Armijo.x, Armijo.xk + Armijo.b**m * Armijo.a0 * Armijo.dk)
        return phi

    def theta(Armijo, m):
        theta = f(Armijo.x, Armijo.xk) + Armijo.w_1 * Armijo.b**m * Armijo.a0 * np.dot(grad_f(Armijo.x, Armijo.xk), Armijo.dk)
        return theta
    
    def fit(Armijo):
        """fit Run until max iteration is reached (not converged) or a respecting Armijo's rule step is found.

        Returns
        -------
        list(int, float): [m, a]
            Returns both iteration steps needed and a respecting Armijo's rule step.
        """
        m = 0
        while(m <= Armijo.max_it):
            if Armijo.phi(m) <= Armijo.theta(m):
                break
            else:
                m += 1

            """if(m==Armijo.max_it):
                print(m)
                print('a didn\'t converged')"""

        a = Armijo.a0 * Armijo.b**(m)
        return [m, a]

In [280]:
class Wolfe():
    """Wolfe is a class implementing the Wolfe's conditions using a bissection method. It has 4 functions in addition to the __init__ function.
    - fit(): is the main function, ensuring the convergency and implementing the bissection algorithm. It calls phi(), phi_grad() and theta() 
    - phi(): is the function to evaluate the first terme of Wolfe's condition. It calls the function f() previously defined by the user.
    - phi_grad(): is the function to evaluate the first terme of Wolfe's condition. It calls the function grad_f() previously defined by the user.
    - theta(): is the function to evaluate the second terme of Wolfe's condition. It calls the functions f() and grad_f() previously defined by the user.

    Notes
    ------
    The class respect a commun pattern for all line search classes. To evaluate descent step call the function fit().

    References
    ------
    J. F. Bonnans, J. C. Gilbert, C. Lemaréchal, C. A. Sagastizábal, 2006. Numerical Optimization: Theoretical and Practical Aspects. https://doi.org/10.5860/choice.41-0357
    https://github.com/scikit-learn/scikit-learn

    Examples
    ------
    >>>goldsteinlinesearch = Goldstein(x, xk, dk, w_1=w_1, w_2=w_2, a_max=a_max, max_it=max_it)
    >>>it, a = goldsteinlinesearch.fit()
    """
    def __init__(
        self,
        x,
        xk,
        dk,
        *,
        w_1=.15,
        w_2=.5,
        a_max=15,
        max_it=50
        ):
        """
        Parameters
        ----------
        x : sympy.IndexedBase
            Previously defined, needed to evaluate phi() and theta()
        xk : np.array (float)
            Coordinates to evaluate the function
        dk : np.array (float)
            direction descent at iteration k
        w_1 : float, optional
            hyperparameter defining Wolfe's first condition, by default .15
        w_2 : float, optional
            hyperparameter defining Wolfe's second condition, by default .5
        a_max : int, optional
            hyperparameter defining maximum step size, by default 15
        max_it : int, optional
            hyperparameter defining maximum of iteration, by default 50
        """
        self.x = x
        self.xk = xk
        self.dk = dk
        self.w_1 = w_1
        self.w_2 = w_2
        self.a_r = a_max
        self.a_l = 0
        self.max_it = max_it
        
    def phi(Wolfe, a):
        phi = f(Wolfe.x, Wolfe.xk + a * Wolfe.dk)
        return phi
    
    def phi_grad(Wolfe, a):
        phi_grad = np.dot(grad_f(Wolfe.x, Wolfe.xk + a*Wolfe.dk), Wolfe.dk)
        return phi_grad

    def theta(Wolfe, a):
        theta = f(Wolfe.x, Wolfe.xk) + Wolfe.w_1 * a * np.dot(grad_f(Wolfe.x, Wolfe.xk), Wolfe.dk)
        return theta

    def fit(Wolfe):
        """
        Calculate a the mean of a window ([a_l, a_r]) and check if c1 is met. If not, shrink the window from the right.
        Then check if c2 is met, if so break, if not, depending on the value (positive or negative of phi_grad(a)) shrink the window from the right or the left.

        Returns
        -------
        list(int, float): [it, a]
            Returns both iteration steps needed and a respecting Goldstein's rule step.

        """
        it = 0
        c1 = Wolfe.phi(Wolfe.a_l)
        c2 = Wolfe.w_2*Wolfe.phi_grad(0)
        
        while(it <= Wolfe.max_it):
            a = (Wolfe.a_l + Wolfe.a_r)/2
            if Wolfe.phi(a) > Wolfe.theta(a) or Wolfe.phi(a) >= c1:
                Wolfe.a_r = a
            else:
                if np.abs(Wolfe.phi_grad(a)) <= - c2:
                    print(f'stop it: {it + 1}')
                    break
                else:
                    if (Wolfe.phi_grad(a) > 0):
                        Wolfe.a_r = a       
                    else:
                        Wolfe.a_l = a
                        c1 = Wolfe.phi(Wolfe.a_l)
            it += 1
            if(it==Wolfe.max_it):
                print(it)
                print('a n\'a pas convergé')
        return [it, a]

In [281]:
class Goldstein():
    """Goldstein is a class implementing the Goldstein's conditions using a bissection method. It has 4 functions in addition to the __init__ function.
    - fit(): is the main function, ensuring the convergency and implementing the bissection algorithm. It calls phi(), phi_grad() and theta() 
    - phi(): is the function to evaluate the first terme of Goldstein's rule. It calls the function f() previously defined by the user.
    - phi_grad(): is the function to evaluate the first terme of Goldstein's rule. It calls the function grad_f() previously defined by the user. (only used to calculate phi'(0))
    - theta(): is the function to evaluate the second terme of Goldstein's rule. It calls the functions f() and grad_f() previously defined by the user.

    Notes
    ------
    The class respect a commun pattern for all line search classes. To evaluate descent step call the function fit().

    References
    ------
    J. F. Bonnans, J. C. Gilbert, C. Lemaréchal, C. A. Sagastizábal, 2006. Numerical Optimization: Theoretical and Practical Aspects. https://doi.org/10.5860/choice.41-0357
    https://github.com/scikit-learn/scikit-learn

    Examples
    ------
    >>>goldsteinlinesearch = Goldstein(x, xk, dk, w_1=w_1, w_2=w_2, a_max=a_max, max_it=max_it)
    >>>it, a = goldsteinlinesearch.fit()
    """
    def __init__(
        self,
        x,
        xk,
        dk,
        *,
        w_1=.15,
        w_2=.5,
        a_max=15,
        max_it=50
        ):
        """
        Parameters
        ----------
        x : sympy.IndexedBase
            Previously defined, needed to evaluate phi() and theta()
        xk : np.array (float)
            Coordinates to evaluate the function
        dk : np.array (float)
            direction descent at iteration k
        w_1 : float, optional
            hyperparameter defining Goldstein's first condition, by default .15
        w_2 : float, optional
            hyperparameter defining Goldstein's second condition, by default .5
        a_max : int, optional
            hyperparameter defining maximum step size, by default 15
        max_it : int, optional
            hyperparameter defining maximum of iteration, by default 50
        """
        self.x = x
        self.xk = xk
        self.dk = dk
        self.w_1 = w_1
        self.w_2 = w_2
        self.a_r = a_max
        self.a_l = 0
        self.max_it = max_it
        
    def phi(Goldstein, a):
        phi = f(Goldstein.x, Goldstein.xk + a * Goldstein.dk)
        return phi
    
    def phi_grad(Goldstein, a):
        phi_grad = np.dot(grad_f(Goldstein.x, Goldstein.xk + a*Goldstein.dk), Goldstein.dk)
        return phi_grad

    def theta(Goldstein, a):
        theta = f(Goldstein.x, Goldstein.xk) + Goldstein.w_1 * a * np.dot(grad_f(Goldstein.x, Goldstein.xk), Goldstein.dk)
        return theta

    def fit(Goldstein):
        """
        Calculate a the mean of a window ([a_l, a_r]) and check if c1 is met. If not, shrink the window from the right.
        Then check if c2 is met, if so break, if not, shrink the window from the left.

        Returns
        -------
        list(int, float): [it, a]
            Returns both iteration steps needed and a respecting Goldstein's rule step.

        """
        it = 0
        c2 = Goldstein.w_2*Goldstein.phi_grad(0)
        phi0 = Goldstein.phi(0)

        while(it <= Goldstein.max_it):
            a = (Goldstein.a_l + Goldstein.a_r)/2
            if Goldstein.phi(a) > Goldstein.theta(a):
                Goldstein.a_r = a
            else:
                if (Goldstein.phi(a) - phi0)/a >= c2:
                    print(f'stop it: {it + 1}')
                    break
                else:
                    Goldstein.a_l = a
            it += 1
            if(it==Goldstein.max_it):
                print('a n\'a pas convergé')
        return [it, a]

In [282]:
class Newton():
    """Newton is a class implementing the Newton-Raphson algorithm using a adaptative learning rate. It has 4 functions in addition to the __init__ function.
    - fit(): is the main function, ensuring the convergency and implementing the Newton method.
    - stop_pos(), stop_grad() and stop_func() are respectively criteria for stopping on position, gradient and function. 
    Using absolute norm to compare all values to epsillon (hyperparameter).

    Notes
    ------
    The class respect a commun pattern for all gradient descent classes. To evaluate descent step call the function fit().

    References
    ------
    J. F. Bonnans, J. C. Gilbert, C. Lemaréchal, C. A. Sagastizábal, 2006. Numerical Optimization: Theoretical and Practical Aspects. https://doi.org/10.5860/choice.41-0357
    https://github.com/scikit-learn/scikit-learn

    Examples
    ------
    >>>Newton = Newton(x, a, x0)
    >>>xk, dk, fk, ak = Newton.fit()
    """
    def __init__(
        self,
        x,
        a,
        x0,
        *,
        stop='gradient',                 #critère d'arrêt sur le gradient, la fonction objectif ou x
        epsilon=.001,
        max_it=50,
        ):
        """
        Parameters
        ----------
        x : sympy.IndexedBase
            symbolic variable managing indexes
        a : class
            hyperparameter calling desired line search method
        x0 : list(float)
            hyperparameter defining initial point for gradient descent
        b : int, optional
            hyperparameter defining learning rate damping factor, by default 1
        stop : str, optional
            hyperparameter defining stopping criteria ("gradient", "position", "function"), by default 'gradient'
        epsilon : float, optional
            hyperparameter defining stopping criteria threshold, by default .001
        max_it : int, optional
            hyperparameter defining maximum of iteration, by default 50
        """
        self.x = x
        self.a = a
        self.x0 = x0
        self.stop = stop
        self.epsilon = epsilon
        self.max_it = max_it
        
        #init
        self.xk = np.array([x0])
        self.fk = np.array([f(x, x0)])
        self.dk = np.array([[np.inf, np.inf]])      #Init at np.inf to ensure not to converge at iteration 1
        self.ak = np.array([])
        

    def stop_pos(Newton):
       if (np.abs(Newton.xk[-1]-Newton.xk[-2]) <= Newton.epsilon).all():
          print(f'xk converged: {np.abs(Newton.xk[-1]-Newton.xk[-2])} < {Newton.epsilon}')
          return True
            
    def stop_grad(Newton):
        if (np.abs(Newton.dk[-1]-Newton.dk[-2]) <= Newton.epsilon).all():
          print(f'grad converged: {np.abs(Newton.dk[-1]-Newton.dk[-2])} < {Newton.epsilon}')
          return True
        
    def stop_func(Newton):
        if (np.abs(Newton.fk[-1]-Newton.fk[-2]) <= Newton.epsilon).all():
          print(f'func converged: {np.abs(Newton.fk[-1]-Newton.fk[-2])} < {Newton.epsilon}')
          return True

    def fit(Newton):
        it=0
        stop_criterion = {"position": Newton.stop_pos, 
                            "gradient": Newton.stop_grad, 
                            "function": Newton.stop_func}
        
        #init
        #compute descent direction
        dk = - np.dot(np.linalg.inv(hess_f(Newton.x, Newton.xk[-1])), grad_f(Newton.x, Newton.xk[-1]))
        #append normalized direction
        Newton.dk = np.append(Newton.dk, [dk/np.linalg.norm(dk)], axis=0)
        #compute new step size
        Newton.ak = np.append(Newton.ak, [Newton.a(Newton.x, Newton.xk[-1], Newton.dk[-1]).fit()[-1]])       #[-1] to get only alpha and not number of it
        #update new point
        Newton.xk = np.append(Newton.xk, [Newton.x0 + Newton.ak[-1]*Newton.dk[-1]], axis=0)
        #evaluate objectif function
        Newton.fk = np.append(Newton.fk, [f(Newton.x, Newton.xk[-1])], axis=0)
        
        while (it <= Newton.max_it):
            dk = - np.dot(np.linalg.inv(hess_f(Newton.x, Newton.xk[-1])), grad_f(Newton.x, Newton.xk[-1]))
            Newton.dk = np.append(Newton.dk, [dk/np.linalg.norm(dk)], axis=0)
            Newton.ak = np.append(Newton.ak, [Newton.a(Newton.x, Newton.xk[-1], Newton.dk[-1]).fit()[-1]])
            Newton.xk = np.append(Newton.xk, [Newton.xk[-1] + Newton.ak[-1]*Newton.dk[-1]], axis=0)
            Newton.fk = np.append(Newton.fk, [f(Newton.x, Newton.xk[-1])], axis=0)

            #convergency test
            if stop_criterion[Newton.stop]():
                print(f'converged at it: {it}')
                break
            
            it += 1
        print(f'not converged')
        return [Newton.xk, Newton.dk, Newton.fk, Newton.ak]

In [283]:
#symbolic variable definition
x = sy.IndexedBase('x')

#Steepest_descent_optimal hyperparameters
a = Wolfe
x0 = [-5, 3]
lim = [[-5, 5], [-5, 5]]

#Steepest_descent_optimal fit
start = time.time()
Newton = Newton(x, a, x0, stop='position')
xk, dk, fk, ak = Newton.fit()

#print output
print(f"t: {time.time() - start} s")
print(f"x*: {xk[-1]}")
print(f"grad*: {dk[-1]}")
print(f"f*: {fk[-1]}")

stop it: 1
stop it: 3
stop it: 6
stop it: 9
stop it: 14
xk converged: [0.00078506 0.00047103] < 0.001
converged at it: 3
not converged
t: 0.2721700668334961 s
x*: [ 3.58067595e-05 -2.14840557e-05]
grad*: [ 0.85749293 -0.51449576]
f*: 8.226107757733654e-10


# 3. Visualization

In [284]:
def visualization(lim, x, xk, fk, x_idx=0, y_idx=1):
    """visualization is a function, using plotly to represente dynamicaly the optimization iterations steps on f. Pandas df are used to facilitate plotly usage.

    Parameters
    ----------
    lim : list(n, 2)
        limite du domaine d'optimisation pour chachune des dimensions
    x : sympy.IndexedBase
        symbolic variable managing indexes.
    xk : nd array(float)
        array of xk iterations
    fk : nd array(float)
        array of evaluation of f at xk iterations
    x_idx : int, optional
        index of the first dimension, by default 0
    y_idx : int, optional
        index of the second dimension, by default 1

    References
    ------
    https://plotly.com/python/

    Examples
    ------
    >>>x_idx = 0
    >>>y_idx = 1
    >>>visualization(lim, x, xk, fk, x_idx, y_idx)
    """
    if x_idx >= np.shape(xk)[1] or y_idx >= np.shape(xk)[1]:
        print('---------- Dimension out of bound, please choose right dimensions ----------')
        sys.exit()
    #Surface plot
    X = np.linspace(lim[0][0], lim[0][1], 100)
    Y = np.linspace(lim[1][0], lim[1][1], 100)
    mesh_X, mesh_Y = np.meshgrid(X, Y)
    Z = f(x,[mesh_X,mesh_Y])

    #gradient descent plot
    grad_desc = pd.DataFrame(xk, columns=['x', 'y'])
    grad_desc['z'] = np.reshape(fk, (-1,1))

    fig = go.Figure()
    fig.add_surface(x=X, y=Y, z=Z)
    fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                    highlightcolor="limegreen", project_z=True))
    fig.update_layout(title='Optimization surface', autosize=False,
                    width=750, height=750
    )
    
    #iterations plot
    fig.add_scatter3d(
        x=grad_desc[grad_desc.columns[x_idx]],
        y=grad_desc[grad_desc.columns[y_idx]],
        z=grad_desc['z'],
        marker=dict(
            size=5,
            color=grad_desc['x'].index,
            colorscale='hot',
        ),
        line=dict(
            color='white',
            width=3
        ))
    
    fig.show()